# Knowledge Distillation process

In this code, we'll apply Knowledge Distillation using a pretrained model (teacher) and a student model. We use the Google's Cloud TPUs in order to train the models and the data used in this project is stored in Google's Cloud Platform. 

## Conecting to Google Cloud Storage (**GCS**)

We use a private bucket to store the data and models, if you want access to this bucket please email us at  cafajar@uis.edu.co.

In [1]:
import uuid
from google.colab import auth

project_id = 'fine-program-318215'
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

auth.authenticate_user()
!gcloud config set project {project_id}

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir folderOnColab
!gcsfuse --implicit-dirs test_cloud_andres folderOnColab

!ls folderOnColab/

Updated property [core/project].
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  76878      0 --:--:-- --:--:-- --:--:-- 79281
OK
38 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.2 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.36.0_amd64.deb ...
Unpacking gcsfuse (0.36.0) ...
Setting up gcsfuse (0.36.0) ...
2021/10/27 19:31:35.918654 Using mount point: /content/folderO

In [2]:
import os
import h5py
import sys
import tempfile
import zipfile
import numpy as np 
import random as rn
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt 
import sklearn.metrics as sklm

#Reproducibility
#seed = 0
#os.environ['PYTHONHASHSEED'] = '0'
#np.random.seed(seed)
#rn.seed(seed)
#tf.random.set_seed(seed)

from tensorflow.keras import backend as K
from tensorflow.keras import optimizers 
from tensorflow.keras import layers
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Enabling the TPU
First, check in the Notebook settings and select TPU from the Hardware Accelerator drop-down.

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')  # TPU detection
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

tpu_strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.90.207.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.207.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Input data
Our input data is stored on Google Cloud Storage. We've stored our input data in TFRecord files. We have five files equally divided to allow for a 
cross-validation training, if needed.

In [4]:
AUTO = tf.data.experimental.AUTOTUNE                    # Allows for optimizations
batch_size = 16 * tpu_strategy.num_replicas_in_sync
fold_no = 1                                             # If not doing cross-validation, 
                                                        # the first set is for validation and the others for training

gcs_pattern = 'gs://test_cloud_andres/tfrecords/11k/kfolds/*.tfrecords'
filenames = tf.io.gfile.glob(gcs_pattern)
validation_fns = filenames.pop(fold_no-1)
train_fns = filenames
test_fns = tf.io.gfile.glob('gs://test_cloud_andres/tfrecords/11k/test_2200_max3.tfrecords')

print('Train TFRecords:',train_fns)
print('Validation TFRecord:',validation_fns)
print('Test TFRecord:',test_fns)

def parse_tfrecord(example):
  features = {'X': tf.io.FixedLenFeature([2049,], tf.float32),  # ECG signal
              'Y': tf.io.FixedLenFeature([1,]   , tf.int64  ),  # class
             }
  example = tf.io.parse_single_example(example, features)
  return example['X'], example['Y']-1

def load_dataset(filenames):
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

train_dataset = load_dataset(train_fns).repeat().shuffle(2000000).batch(batch_size).prefetch(AUTO) 
val_dataset   = load_dataset(validation_fns).batch(batch_size).prefetch(AUTO) 
test_dataset  = load_dataset(test_fns).batch(batch_size).prefetch(AUTO) 

Train TFRecords: ['gs://test_cloud_andres/tfrecords/11k/kfolds/train_1760_max3_f2.tfrecords', 'gs://test_cloud_andres/tfrecords/11k/kfolds/train_1760_max3_f3.tfrecords', 'gs://test_cloud_andres/tfrecords/11k/kfolds/train_1760_max3_f4.tfrecords', 'gs://test_cloud_andres/tfrecords/11k/kfolds/train_1760_max3_f5.tfrecords']
Validation TFRecord: gs://test_cloud_andres/tfrecords/11k/kfolds/train_1760_max3_f1.tfrecords
Test TFRecord: ['gs://test_cloud_andres/tfrecords/11k/test_2200_max3.tfrecords']


Calculating steps for training

In [5]:
"""
The number of signals in each TFRecord file was previously calculated and is
hard-coded in this cell to avoid loading the data (expensive).
"""

test_size = 507443
test_steps = int(np.ceil(test_size/batch_size))

def get_steps(fold_no, batch_size):
  total_size = 2048149
  if fold_no == 1:
    val_size = 410780
    return  int(np.ceil((total_size - val_size)/batch_size)), int(np.ceil(val_size/batch_size))
  elif fold_no == 2:
    val_size = 410539
    return  int(np.ceil((total_size - val_size)/batch_size)), int(np.ceil(val_size/batch_size))
  elif fold_no == 3:
    val_size = 409318
    return  int(np.ceil((total_size - val_size)/batch_size)), int(np.ceil(val_size/batch_size))
  elif fold_no == 4:
    val_size = 407967
    return  int(np.ceil((total_size - val_size)/batch_size)), int(np.ceil(val_size/batch_size))
  elif fold_no == 5:
    val_size = 409545
    return  int(np.ceil((total_size - val_size)/batch_size)), int(np.ceil(val_size/batch_size))

train_steps, val_steps = get_steps(fold_no, batch_size)

print(train_steps)
print(val_steps)
print(test_steps)

12792
3210
3965


Loading ground-truth values to memory for latter evaluation


In [6]:
y_true = []
for signal in test_dataset:
  Y = signal[1].numpy()
  y_true.extend(Y)
y_true = np.array(y_true)

## Teacher Model
We used several models as teachers, for more details see our paper. In this case we use the model with 7,093 parameters as teacher.

In [8]:
with tpu_strategy.scope():  # Model is created in the TPUStrategy so it will train on the TPU
  def zeropad(x, filters):  # Pad zeros to match dimensions
    pad = K.zeros_like(x)
    assert (filters % pad.shape[2]) == 0
    num_repeat = filters // pad.shape[2]
    for i in range(num_repeat - 1):
        x = K.concatenate([x, pad], axis=2)
    return x 

  def basic_block(x_in, pool_size, strides, filters, kernel_size, DP):
      y = layers.MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(x_in)
      y = layers.Lambda(zeropad, arguments={'filters':filters})(y) 

      x = layers.BatchNormalization(axis=-1)(x_in)
      x = layers.ReLU()(x) 
      x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
      x = layers.BatchNormalization(axis=-1)(x)
      x = layers.ReLU()(x)
      x = layers.Dropout(DP)(x)
      x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
      x = layers.AveragePooling1D(pool_size=pool_size, strides=strides, padding='same')(x)
      x = layers.Add()([y,x])
      return x

  # Training parameters
  res_blocks  = 13
  initial_filters = 2
  s_j = 8

  kernel_size = 16
  input_shape = (2049, 1)
  DP = 0.2
  pool_size = 2
  strides = 2
  k = 0

  ##############################################################################
  ################################# MODEL ######################################

  filters = initial_filters*(2**k) # Modify the outputs of the conv layers
  input_signal = tf.keras.Input(shape=input_shape, name='ECG_signal')
  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(input_signal)
  x = layers.BatchNormalization(axis=-1)(x)
  x = layers.ReLU()(x)

  y = layers.MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(x)
  y = layers.Lambda(zeropad, arguments={'filters':filters})(y) 

  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
  x = layers.BatchNormalization(axis=-1)(x)
  x = layers.ReLU()(x)
  x = layers.Dropout(DP)(x)
  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
  x = layers.Add()([y,x])

  for i in range(res_blocks):
      if i%s_j == 0:
          filters = initial_filters*(2**k)
          k = k + 1 
          strides = 2
          x = basic_block(x, pool_size, strides, filters, kernel_size, DP)
      else:
          strides = 1  
          x = basic_block(x, pool_size, strides, filters, kernel_size, DP)

  x = layers.BatchNormalization(axis=-1)(x)        
  x = layers.ReLU()(x)
  x = layers.Flatten()(x)
  outputs = layers.Dense(3)(x)
  teacher = tf.keras.Model(inputs=input_signal, outputs=outputs)
  teacher.compile(
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
      steps_per_execution = 2400  # between 2 and steps_per_epoch
      )

In [9]:
teacher.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ECG_signal (InputLayer)         [(None, 2049, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 2049, 2)      34          ECG_signal[0][0]                 
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 2049, 2)      8           conv1d_19[0][0]                  
__________________________________________________________________________________________________
re_lu_19 (ReLU)                 (None, 2049, 2)      0           batch_normalization_19[0][0]     
____________________________________________________________________________________________

In [11]:
teacher.load_weights('folderOnColab/models/deep_models/KD/KD_7k_distilled.h5')

## Evaluating pre-trained teacher

In [ ]:
(loss, acc_full) = teacher.evaluate(val_dataset, steps=val_steps, verbose=1)
(loss, acc_full) = teacher.evaluate(test_dataset, steps=test_steps,  verbose=1)

3965/3965 [==============================] - 21s 5ms/step - loss: 0.2236 - sparse_categorical_accuracy: 0.9137


In [ ]:
y_pred = teacher.predict(test_dataset, steps=test_steps, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_true_for_f1 = y_true

print(classification_report(y_true_for_f1, y_pred_bool))
print('acc',accuracy_score(y_true_for_f1, y_pred_bool))
print('precision',precision_score(y_true_for_f1, y_pred_bool , average="macro"))
print('recall',recall_score(y_true_for_f1, y_pred_bool , average="macro"))
print('f1',f1_score(y_true_for_f1, y_pred_bool , average="macro"))

3965/3965 [==============================] - 38s 10ms/step
              precision    recall  f1-score   support

           0       0.91      0.95      0.93    287147
           1       0.74      0.84      0.78     17369
           2       0.93      0.87      0.90    202927

    accuracy                           0.91    507443
   macro avg       0.86      0.89      0.87    507443
weighted avg       0.92      0.91      0.91    507443

acc 0.9137262707338558
precision 0.8609209052594092
recall 0.885844347626335
f1 0.8718258185795614


Also check teacher size

In [13]:
def get_gzipped_model_size(model):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, keras_file = tempfile.mkstemp('.h5')
  model.save(keras_file, include_optimizer=False)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)

  return os.path.getsize(zipped_file)

print(get_gzipped_model_size(teacher), 'bytes')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


47487 bytes


## Student Model
We used several models as students, for more details see our paper. In this case we use the model with 4,455 parameters as student.

In [14]:
with tpu_strategy.scope():  # Model is created in the TPUStrategy so it will train on the TPU
  def zeropad(x, filters):  # Pad zeros to match dimensions
    pad = K.zeros_like(x)
    assert (filters % pad.shape[2]) == 0
    num_repeat = filters // pad.shape[2]
    for i in range(num_repeat - 1):
        x = K.concatenate([x, pad], axis=2)
    return x 

  def basic_block(x_in, pool_size, strides, filters, kernel_size, DP):
      y = layers.MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(x_in)
      y = layers.Lambda(zeropad, arguments={'filters':filters})(y) 

      x = layers.BatchNormalization(axis=-1)(x_in)
      x = layers.ReLU()(x) 
      x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
      x = layers.BatchNormalization(axis=-1)(x)
      x = layers.ReLU()(x)
      x = layers.Dropout(DP)(x)
      x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
      x = layers.AveragePooling1D(pool_size=pool_size, strides=strides, padding='same')(x)
      x = layers.Add()([y,x])
      return x

  # Training parameters
  res_blocks  = 8
  initial_filters = 2
  s_j = 8

  kernel_size = 16
  input_shape = (2049, 1)
  DP = 0.2
  pool_size = 2
  strides = 2
  k = 0

  ##############################################################################
  ################################# MODEL ######################################

  filters = initial_filters*(2**k) # Modify the outputs of the conv layers
  input_signal = tf.keras.Input(shape=input_shape, name='ECG_signal')
  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(input_signal)
  x = layers.BatchNormalization(axis=-1)(x)
  x = layers.ReLU()(x)

  y = layers.MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(x)
  y = layers.Lambda(zeropad, arguments={'filters':filters})(y) 

  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
  x = layers.BatchNormalization(axis=-1)(x)
  x = layers.ReLU()(x)
  x = layers.Dropout(DP)(x)
  x = layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
  x = layers.Add()([y,x])

  for i in range(res_blocks):
      if i%s_j == 0:
          filters = initial_filters*(2**k)
          k = k + 1 
          strides = 2
          x = basic_block(x, pool_size, strides, filters, kernel_size, DP)
      else:
          strides = 1  
          x = basic_block(x, pool_size, strides, filters, kernel_size, DP)

  x = layers.BatchNormalization(axis=-1)(x)        
  x = layers.ReLU()(x)
  x = layers.Flatten()(x)
  outputs = layers.Dense(3)(x)
  student = tf.keras.Model(inputs=input_signal, outputs=outputs)
  student.compile(
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
      steps_per_execution = 2400  # between 2 and steps_per_epoch
      )

In [15]:
student.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ECG_signal (InputLayer)         [(None, 2049, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_48 (Conv1D)              (None, 2049, 2)      34          ECG_signal[0][0]                 
__________________________________________________________________________________________________
batch_normalization_48 (BatchNo (None, 2049, 2)      8           conv1d_48[0][0]                  
__________________________________________________________________________________________________
re_lu_48 (ReLU)                 (None, 2049, 2)      0           batch_normalization_48[0][0]     
____________________________________________________________________________________________

## Knowledge Distillation process
Here we show the distillation process for the models created above. We tested several values of *Temperature* and *Alpha*. In this case, we used 70 and 0.2, respectively.

### Distiller model

In [16]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:

            temperature = 70
            alfa = 0.2

            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.compiled_loss(y, student_predictions, regularization_losses=self.losses)
            distillation_loss_fn = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)
            distillation_loss = distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / temperature, axis=1),
                tf.nn.softmax(student_predictions / temperature, axis=1),
            )
            loss = alfa * student_loss + (1 - alfa) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        self.compiled_loss(y, y_prediction, regularization_losses=self.losses)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        return {m.name: m.result() for m in self.metrics}

In [17]:
with tpu_strategy.scope():
  distiller = Distiller(teacher=teacher, student=student)
  distiller.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE),
      steps_per_execution=2400,
  )

### Training the distiller model

In [ ]:
callbacks_distilled = [
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-3,
    patience=10,
    verbose=1,
    restore_best_weights=True),    
tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.1,
    patience=3,
    verbose=1,
    min_lr=0.00000000001),
tf.keras.callbacks.ModelCheckpoint(
    filepath='folderOnColab/models/deep_models/KD_T7k_S4k.h5',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True,
    save_weights_only=True),
]

history_student_distilled = distiller.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100, 
    steps_per_epoch=train_steps,
    validation_steps=val_steps,
    callbacks=callbacks_distilled, 
    )

In [ ]:
distiller.save_weights('folderOnColab/models/deep_models/KD/KD_T7k_S4k.h5')
distiller.student.save_weights('folderOnColab/models/deep_models/KD/KD_4k_distilled.h5')

### Evaluating the distiller model

In [ ]:
(loss, acc_full) = distiller.evaluate(val_dataset, steps=val_steps, verbose=1)
(loss, acc_full) = distiller.evaluate(test_dataset, steps=test_steps,  verbose=1)

3965/3965 [==============================] - 21s 5ms/step - loss: 0.2755 - sparse_categorical_accuracy: 0.8924


In [ ]:
y_pred = distiller.student.predict(test_dataset, steps=test_steps, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_true_for_f1 = y_true

print(classification_report(y_true_for_f1, y_pred_bool))
print('acc',accuracy_score(y_true_for_f1, y_pred_bool))
print('precision',precision_score(y_true_for_f1, y_pred_bool , average="macro"))
print('recall',recall_score(y_true_for_f1, y_pred_bool , average="macro"))
print('f1',f1_score(y_true_for_f1, y_pred_bool , average="macro")) 

3965/3965 [==============================] - 27s 7ms/step
              precision    recall  f1-score   support

           0       0.89      0.93      0.91    287147
           1       0.64      0.66      0.65     17369
           2       0.92      0.85      0.88    202927

    accuracy                           0.89    507443
   macro avg       0.81      0.82      0.82    507443
weighted avg       0.89      0.89      0.89    507443

acc 0.8924312681424318
precision 0.81473504904581
recall 0.8170367378006674
f1 0.815291889378193


In [22]:
print(get_gzipped_model_size(student), 'bytes')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


31502 bytes
